In [ ]:
# ✅ Versione compatibile con Google Colab con valori numerici e heatmap + visualizzazione Q, K, V
# 📘 Visualizza la matrice di attenzione BERT con token ricostruiti e valori colorati

# ⚙️ Installa pacchetti richiesti (Colab ha già tensorflow)
!pip install -q transformers matplotlib

# 📚 Import
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from transformers import TFAutoModel, AutoTokenizer

# 🔁 Caricamento modello BERT multilingua
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name, output_attentions=True, output_hidden_states=True)

# 🧠 Funzione per ricostruire i token originari
def ricostruisci_parole(tokens):
    parole = []
    parola = ""
    for tok in tokens:
        if tok.startswith("##"):
            parola += tok[2:]
        else:
            if parola:
                parole.append(parola)
            parola = tok
    if parola:
        parole.append(parola)
    return parole

# 🔍 Visualizzazione matrice di attenzione con heatmap, numeri e Q/K/V
def visualizza_attention_con_bert(frase):
    inputs = tokenizer(frase, return_tensors="tf")
    outputs = model(**inputs, training=False)

    # Ultimo layer, head 0
    attention = outputs.attentions[-1][0][0].numpy()
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    parole = ricostruisci_parole(tokens)

    print("\nToken BERT:", tokens)
    print("Token ricostruiti:", parole)

    # Visualizza Q, K, V (solo se disponibili via output_hidden_states)
    hidden_states = outputs.hidden_states[-1][0].numpy()  # shape: [seq_len, hidden_dim]
    print("\n🔹 Dimensione embedding finale:", hidden_states.shape)

    # Matrici Query, Key, Value simulate (solo per visualizzazione, non reali come nei pesi del modello)
    Q = hidden_states @ np.random.rand(hidden_states.shape[1], hidden_states.shape[1])
    K = hidden_states @ np.random.rand(hidden_states.shape[1], hidden_states.shape[1])
    V = hidden_states @ np.random.rand(hidden_states.shape[1], hidden_states.shape[1])

    print("\n📌 Esempio dimensioni Q, K, V:")
    print("Q:", Q.shape, "\nK:", K.shape, "\nV:", V.shape)

    # Heatmap
    plt.figure(figsize=(10, 8))
    im = plt.imshow(attention, cmap="viridis")
    plt.title("Matrice di Attenzione BERT (Ultimo layer, Head 0)")
    plt.xlabel("Token osservato (Key)")
    plt.ylabel("Token osservatore (Query)")
    plt.xticks(np.arange(len(tokens)), tokens, rotation=45)
    plt.yticks(np.arange(len(tokens)), tokens)
    plt.colorbar(im, label="Peso di attenzione")

    # Numeri nella heatmap
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            val = attention[i, j]
            plt.text(j, i, f"{val:.2f}", ha="center", va="center",
                     color="white" if val < 0.5 else "black", fontsize=8)

    plt.tight_layout()
    plt.show()

# ✏️ Inserisci la frase da analizzare qui:
visualizza_attention_con_bert("Il gatto mangia il topo.")
